# 9-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

cloud jupyter 를 열고 aiffel/recommendata_iu/ 디렉토리에 python3로 ipynb파일을 생성 해 주세요 jupyter notebook 환경에서 프로젝트 코드를 작성하고 제출하시면 됩니다.

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수** 로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결 해 줍니다.

Cloud shell에서 아래 명령어를 입력해주세요.

- mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
- ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

목차
---
1) 데이터 준비와 전처리.  
2) 분석해 봅시다.  
3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.  
4) CSR matrix를 직접 만들어 봅시다.  
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.  
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다

## 1) 데이터 준비와 전처리.
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [164]:
# 필요한 모듈 import
import numpy as np
import pandas as pd
import os

In [165]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [166]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


3점 이상인 영화의 비율이 83.63% 임을 알 수 있습니다.

In [167]:
# rating 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [168]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [170]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [171]:
# 영화 제목 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.
movies['title'] = movies['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


In [172]:
# ratings 와 movies 데이터 프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy


In [173]:
# user_id가 1인 사용자가 어떤 영화를 시청했는지 확인해봅니다.
movie_user_id_1 = data[data['user_id']==1]['title']
movie_user_id_1

0                one flew over the cuckoo's nest (1975)
1                      james and the giant peach (1996)
2                                   my fair lady (1964)
3                                erin brockovich (2000)
4                                  bug's life, a (1998)
5                            princess bride, the (1987)
6                                        ben-hur (1959)
7                             christmas story, a (1983)
8                snow white and the seven dwarfs (1937)
9                              wizard of oz, the (1939)
10                          beauty and the beast (1991)
11                                          gigi (1958)
12                        miracle on 34th street (1947)
13                      ferris bueller's day off (1986)
14                           sound of music, the (1965)
15                                     airplane! (1980)
16                                        tarzan (1999)
17                                         bambi

## 2) 분석해 봅시다.
---
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

### 2-1. ratings에 있는 유니크한 영화 개수

In [174]:
ratings['movie_id'].nunique()

3628

### 2-2. ratings에 있는 유니크한 사용자 수

In [175]:
ratings['user_id'].nunique()

6039

### 2-3. 가장 인기 있는 영화 30개(인기순)

In [177]:
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2510
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2435
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [178]:
# 유저별 몇 편의 영화를 시청했는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6040.000000
mean      138.490563
std       156.238108
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
---

우선 keyword로 영화를 찾고 movie_id를 이용해서 영화를 찾습니다.

In [179]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('terminator', regex=False)]

,movie_id,title,genre
585,589,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller
1220,1240,"terminator, the (1984)",Action|Sci-Fi|Thriller


In [180]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('shawshank', regex=False)]

,movie_id,title,genre
315,318,"shawshank redemption, the (1994)",Drama


In [181]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('matrix', regex=False)]

,movie_id,title,genre
2502,2571,"matrix, the (1999)",Action|Sci-Fi|Thriller


In [182]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('forrest', regex=False)]

,movie_id,title,genre
352,356,forrest gump (1994),Comedy|Romance|War


In [183]:
# keyword 로 영화 찾기
movies[movies['title'].str.lower().str.contains('toy', regex=False)]

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1948,2017,babes in toyland (1961),Children's|Fantasy|Musical
2184,2253,toys (1992),Action|Comedy|Fantasy
2411,2480,dry cleaning (nettoyage à sec) (1997),Drama
3017,3086,march of the wooden soldiers (a.k.a. laurel & ...,Comedy
3045,3114,toy story 2 (1999),Animation|Children's|Comedy


In [185]:
# 내가 선호하는 영화 5개 선정
# (한글)영화 이름 /(영어) 영화 이름 / movie_id
# 쇼생크 탈출 / shawshank redemption, the (1994) / 318
# 매트릭스 / matrix, the (1999) / 2571
# 포레스트 검프 / forrest gump (1994) / 356
# 터미네이터 2 / terminator 2: judgment day (1991) / 589
# 토이스토리 / toy story (1995) / 1

# 내가 선호하는 영화 id 리스트
my_favorite_movie_id = [318, 2571, 356 , 589, 1]
my_favorite = ['shawshank redemption, the (1994)',
              'matrix, the (1999)',
              'forrest gump (1994)',
              'terminator 2: judgment day (1991)',
              'toy story (1995)']

# 'hojin'이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['6100']*5,
                             'movie_id': my_favorite_movie_id,
                             'title': my_favorite,
                             'counts':[5, 4, 3, 5, 5]})

if not data.isin({'user_id':['hojin']})['user_id'].any():  # user_id에 'hojin'이라는 데이터가 없다면
    data = data.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data = data[['user_id', 'movie_id', 'title', 'counts']]
data.tail()

KeyError: "['counts'] not in index"

## 4) CSR matrix를 직접 만들어 봅시다.
---

In [153]:
# 고유한 유저, 영화를 찾기
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

In [154]:
# 유저, 영화 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [155]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['new_user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['new_movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing OK!!


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


title column indexing OK!!


,user_id,movie_id,title,count,new_user_id,new_movie_id
0,1,1193,one flew over the cuckoo's nest (1975),NaN,0,0
1,1,661,james and the giant peach (1996),NaN,0,1
2,1,914,my fair lady (1964),NaN,0,2
3,1,3408,erin brockovich (2000),NaN,0,3
4,1,2355,"bug's life, a (1998)",NaN,0,4
...,...,...,...,...,...,...
0,hojin,318,"shawshank redemption, the (1994)",5.0,6039,157
1,hojin,2571,"matrix, the (1999)",5.0,6039,124
2,hojin,356,forrest gump (1994),5.0,6039,160
3,hojin,589,terminator 2: judgment day (1991),5.0,6039,92


In [156]:
# CSR Matrix 생성
num_user = data['new_user_id'].nunique()
num_movie = data['new_movie_id'].nunique()

# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_data = csr_matrix((data['count'], (data.new_user_id, data.new_movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
---

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습시킵니다.

- implicit 패키지는 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉짱히 빠르게 학습할 수 있는 패키지입니다.
- 이 패키지에서 구현된 als(AlternatingLeastSquares) 모델을 사용합니다.

In [157]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보면,

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

참고로 1,4를 늘릴수록 학습을 잘 하게 되지만 과적합의 우려가 있습니다.

In [158]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [159]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose 해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [160]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

ModelFitError: NaN encountered in factors

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
---

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
---

In [163]:
hojin, matrix = user_to_idx['hojin'], movie_to_idx['shawshank redemption']
hojin_vector, matrix_vector = als_model.user_factors[lil_park], als_model.item_factors[matrix]

KeyError: 'shawshank redemption'

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
---

루브릭

1. CSR matrix가 정상적으로 만들어졌다.

- 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

- 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

- MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.